In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import copy

In [2]:
dataset = pd.read_csv('data.csv')
# dataset

In [3]:
X = dataset.drop('xAttack', axis = 1)
Y = dataset['xAttack']

In [4]:
X = (X - np.mean(X))/np.std(X)
X.shape

(125973, 29)

In [5]:
X = X.iloc[:]
# X_copy = copy.deepcopy(X_train)
# Y_copy = copy.deepcopy(Y_train)
# Y_train
Y=pd.concat([Y,pd.get_dummies(Y,prefix='xAttack')],axis=1).drop(['xAttack'],axis=1)
Y = Y.iloc[:]
# X

In [6]:
class Neural_network(object):
    
    def __init__(self):
        self.input_layer_size = 29
        self.output_layer_size = 29
        self.hidden_layer_size = 14
        self.hidden_layers = 1
        self.weights = {}
        self.dweights = {}
        self.Z = {}
        self.epoch = 50
        self.ita = 0.001
        self.a = {}
        self.Y_hat = np.array([])
        self.delta = {}
        
    def initialization(self):
        
        self.weights[1] = np.random.randn(self.input_layer_size,self.hidden_layer_size)*0.01
        self.weights[self.hidden_layers+1] = np.random.randn(self.hidden_layer_size,self.output_layer_size)*0.01
        
        for i in np.arange(2,self.hidden_layers+1):
            self.weights[i] = np.random.randn(self.hidden_layer_size,self.hidden_layer_size)*0.01
            
    
    def softmax(self,a):
        tsum = np.sum(a,axis = 1,keepdims = True)
        return np.divide(a,tsum)
    
    
    def error(self,y):
#         return sum(0.5*np.sum((y-self.Y_hat)*(y-self.Y_hat),axis = 1))/y.shape[0]
        return np.mean(0.5*np.sum((y-self.Y_hat)*(y-self.Y_hat),axis = 1))
    
    def derivative_sigmoid(self,layer):
        derivative = layer*(1-layer)
        return derivative
    
   
    def deltas(self,y):
        w = self.a[self.hidden_layers+2]
        error = (y - self.Y_hat)
        self.delta[self.hidden_layers+2] = -1 * error * self.derivative_sigmoid(w)
        xx = self.a[self.hidden_layers+1].T
        yy = self.delta[self.hidden_layers+2]
        self.dweights[self.hidden_layers+1] = np.dot(xx , yy)
        
        for i in np.arange(self.hidden_layers+1,1,-1):
            aa = self.delta[i+1]
            bb = self.weights[i].T
            deri = self.derivative_sigmoid(self.a[i])
            self.delta[i] = ( np.dot(aa, bb))*deri
            self.dweights[i-1] = np.dot(self.a[i-1].T , (self.delta[i]))  
    
    def sigmoid(self,x):
        x = -x
        return 1 / (1 + np.exp(x))
    
    
    
    def forward_propagation(self, X):
        self.a[1]=np.array(X)
        
        fg = 1
        for i in range(1,self.hidden_layers+2):
            if fg:
                self.Z[i+1] = np.dot(self.a[i],self.weights[i])
                self.a[i+1] = self.sigmoid(self.Z[i+1])
        self.Y_hat = self.softmax(self.a[self.hidden_layers+2])
        
   
    def gradient_descent(self,X_train,Y_train):

        k = 0
        while k <= 120000:
            for i in range(self.epoch):
                Y = Y_train[k:k+5000]
                X = X_train[k:k+5000]
                self.forward_propagation(X)
                self.deltas(Y)
                for j in range(1,self.hidden_layers+2):
                    lrweights = self.ita*self.dweights[j]
                    self.weights[j] = self.weights[j] - lrweights
            print("Final Error : ",self.error(Y))
            k = k + 5000

        
    def predict(self,X):
        self.a[1] = np.array(X)
        self.Z[1] = np.dot(self.a[1],self.weights[1])
        return self.sigmoid(self.Z[1])
    
            

In [7]:
k = Neural_network()
k.initialization()
k.gradient_descent(X,X)

Final Error :  11.539338677043755
Final Error :  11.15410714309359
Final Error :  11.638342590265866
Final Error :  11.76200956637307
Final Error :  13.227533741640201
Final Error :  11.876677549124041
Final Error :  11.936809521589081
Final Error :  12.003756607050269
Final Error :  16.64922127657475
Final Error :  11.703793594540524
Final Error :  10.244871765737617
Final Error :  29.535664753579557
Final Error :  24.125299023280675
Final Error :  12.753684115628127
Final Error :  11.019221304208981
Final Error :  11.442484632160246
Final Error :  12.565600452979501
Final Error :  11.572774949240015
Final Error :  12.603477254391011
Final Error :  11.272049154433065
Final Error :  12.124757819889474
Final Error :  10.918268489078702
Final Error :  11.876262464330482
Final Error :  11.250033902145633
Final Error :  12.44982609958721


In [8]:
# X_test = X_test[0:1000]
# Y_test = Y_test[0:1000]

finaldata=k.predict(X)
finaldata=pd.DataFrame(finaldata)
df_final = pd.concat([finaldata,dataset['xAttack']],axis=1)
# df_final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,xAttack
0,7.502101e-01,0.998322,0.625458,6.725594e-01,0.104884,9.936196e-01,9.185078e-01,0.785174,7.755980e-01,0.818181,0.785373,7.517214e-01,0.982370,8.158696e-01,normal
1,2.016271e-06,0.944586,0.020868,9.718756e-01,0.884068,9.991088e-01,7.968231e-01,0.999276,9.826145e-01,0.822725,0.934223,2.243070e-04,0.000261,3.784907e-01,normal
2,7.955386e-01,0.997234,0.020402,9.998314e-01,0.922336,3.246703e-03,6.135542e-01,0.185471,7.235519e-01,0.014330,0.002920,8.388727e-01,0.004132,9.795604e-01,dos
3,8.885023e-01,0.016840,0.964016,9.454040e-01,0.005800,9.157241e-01,4.190991e-02,0.247402,9.312612e-01,0.794714,0.693977,9.028951e-01,0.941738,7.820054e-01,normal
4,9.880269e-01,0.002106,0.987487,4.142747e-01,0.907618,9.917652e-01,5.231305e-01,0.064475,9.873575e-01,0.876617,0.249446,7.036971e-01,0.946343,4.598728e-01,normal
5,9.878430e-01,0.243114,0.068802,2.177491e-05,0.956339,1.270725e-05,7.831044e-01,0.988761,8.206547e-08,0.999289,1.000000,9.878008e-01,0.005381,1.896777e-01,dos
6,8.117890e-01,0.997955,0.015022,9.997255e-01,0.934330,4.300075e-03,8.520761e-01,0.253083,5.761053e-01,0.009915,0.002000,9.585588e-01,0.004034,9.788813e-01,dos
7,8.003480e-01,0.998536,0.035882,9.997388e-01,0.904106,3.745675e-03,6.246800e-01,0.279022,7.332640e-01,0.008711,0.001839,7.938614e-01,0.004107,9.739320e-01,dos
8,8.396645e-01,0.984775,0.019208,9.991752e-01,0.966436,7.478233e-03,9.825260e-01,0.320052,3.520039e-01,0.003677,0.000558,9.986111e-01,0.002112,9.685290e-01,dos
9,8.003299e-01,0.998326,0.019546,9.997907e-01,0.918038,3.707096e-03,7.133429e-01,0.252629,6.725010e-01,0.011877,0.002528,8.789491e-01,0.003972,9.793076e-01,dos


In [9]:
df_final.to_csv("data_final.csv",index=False)